# Load and Clean Dataset

## Purpose:
This notebook covers the Prepare + Process stages of the analytics lifecycle.

## Objectives:

- Load raw data from data_raw/ (excluded from Git)

- Standardise column names

- Validate and clean data (types, missing values, duplicates)

- Save a cleaned dataset into data_cleaned/ for analysis in Notebook 03

Dataset: Amazon Top 50 Bestselling Books (2009–2019)

In [18]:
from pathlib import Path
import re

import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)


In [19]:
# Robust repo root detection: walk upwards until we find requirements.txt
here = Path.cwd()
repo_root = next((p for p in [here, *here.parents] if (p / "requirements.txt").exists()), None)

if repo_root is None:
    raise FileNotFoundError(
        "Repo root not found. Ensure requirements.txt exists at the repo root."
    )

print("Working directory:", here)
print("Repo root:", repo_root)

# Define file paths (always repo-root relative)
raw_path = repo_root / "data_raw" / "amazon_books_raw.csv"
clean_dir = repo_root / "data_cleaned"
clean_path = clean_dir / "amazon_books_cleaned.csv"

print("Raw path:", raw_path)
print("Clean output path:", clean_path)



Working directory: /workspaces/london-airbnb-market-expansion/notebooks
Repo root: /workspaces/london-airbnb-market-expansion
Raw path: /workspaces/london-airbnb-market-expansion/data_raw/amazon_books_raw.csv
Clean output path: /workspaces/london-airbnb-market-expansion/data_cleaned/amazon_books_cleaned.csv


In [20]:
if not raw_path.exists():
    raise FileNotFoundError(
        f"Raw file not found at: {raw_path}\n"
        "Ensure the CSV is located in data_raw/ and named amazon_books_raw.csv."
    )

print("Raw file found.")



Raw file found.


In [21]:
df_raw = pd.read_csv(raw_path)

df_raw.head()


df_raw.shape

df_raw.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         550 non-null    object 
 1   Author       550 non-null    object 
 2   User Rating  550 non-null    float64
 3   Reviews      550 non-null    int64  
 4   Price        550 non-null    int64  
 5   Year         550 non-null    int64  
 6   Genre        550 non-null    object 
dtypes: float64(1), int64(3), object(3)
memory usage: 30.2+ KB


In [22]:
def to_snake_case(col: str) -> str:
    col = col.strip()
    col = re.sub(r"[^\w\s]", "", col)   # remove punctuation
    col = re.sub(r"\s+", "_", col)      # spaces -> underscore
    return col.lower()

df = df_raw.copy()
df.columns = [to_snake_case(c) for c in df.columns]

df.columns.tolist()




['name', 'author', 'user_rating', 'reviews', 'price', 'year', 'genre']

This project expects the following fields:

- name

- author

- genre

- user_rating

- reviews

- price

- year

If the dataset uses different names, we will map them before continuing.

In [23]:
required_cols = ["name", "author", "genre", "user_rating", "reviews", "price", "year"]
missing = [c for c in required_cols if c not in df.columns]

if missing:
    raise ValueError(
        f"Missing expected columns: {missing}\n"
        f"Available columns: {list(df.columns)}\n"
        "Fix by mapping/renaming columns before proceeding."
    )

df[required_cols].head()



,name,author,genre,user_rating,reviews,price,year
0,10-Day Green Smoothie Cleanse,JJ Smith,Non Fiction,4.7,17350,8,2016
1,11/22/63: A Novel,Stephen King,Fiction,4.6,2052,22,2011
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,Non Fiction,4.7,18979,15,2018
3,1984 (Signet Classics),George Orwell,Fiction,4.7,21424,6,2017
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,Non Fiction,4.8,7665,12,2019


## Initial Data Quality Checks 

- Missing values

- Duplicate rows

- Basic descriptive statistics

In [24]:
df.isna().sum().sort_values(ascending=False)

df.duplicated().sum()

df.describe(include="all").T


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
name,550,351,Publication Manual of the American Psychologic...,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
author,550,248,Jeff Kinney,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
user_rating,550.0,NaN,NaN,NaN,4.618364,0.22698,3.3,4.5,4.7,4.8,4.9
reviews,550.0,NaN,NaN,NaN,11953.281818,11731.132017,37.0,4058.0,8580.0,17253.25,87841.0
price,550.0,NaN,NaN,NaN,13.1,10.842262,0.0,7.0,11.0,16.0,105.0
year,550.0,NaN,NaN,NaN,2014.0,3.165156,2009.0,2011.0,2014.0,2017.0,2019.0
genre,550,2,Non Fiction,310,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Data Type Cleaning

- year → integer

- price → numeric

- reviews → integer

- user_rating → numeric

In [25]:
def to_numeric_safe(series: pd.Series) -> pd.Series:
    return pd.to_numeric(
        series.astype(str).str.replace(",", "", regex=False).str.strip(),
        errors="coerce"
    )

df["year"] = to_numeric_safe(df["year"]).astype("Int64")
df["price"] = to_numeric_safe(df["price"])
df["reviews"] = to_numeric_safe(df["reviews"]).astype("Int64")
df["user_rating"] = to_numeric_safe(df["user_rating"])

df[["year", "price", "reviews", "user_rating"]].describe()


,year,price,reviews,user_rating
count,550.0,550.000000,550.0,550.000000
mean,2014.0,13.100000,11953.281818,4.618364
std,3.165156,10.842262,11731.132017,0.226980
min,2009.0,0.000000,37.0,3.300000
25%,2011.0,7.000000,4058.0,4.500000
50%,2014.0,11.000000,8580.0,4.700000
75%,2017.0,16.000000,17253.25,4.800000
max,2019.0,105.000000,87841.0,4.900000


## Text Cleaning

In [26]:
df["name"] = df["name"].astype(str).str.strip()
df["author"] = df["author"].astype(str).str.strip()
df["genre"] = df["genre"].astype(str).str.strip().str.title()

df["genre"].value_counts(dropna=False)

genre
Non Fiction    310
Fiction        240
Name: count, dtype: int64

## Missing Values & Duplicates Handling

In [27]:
before = df.shape[0]

df_clean = df.dropna(subset=required_cols).copy()
df_clean = df_clean.drop_duplicates()

after = df_clean.shape[0]
print("Rows before:", before)
print("Rows after :", after)
print("Removed    :", before - after)



Rows before: 550
Rows after : 550
Removed    : 0


## Validation Rules (Markdown)

- user_rating between 0 and 5

- price non-negative

- reviews non-negative

In [28]:
df_clean = df_clean[
    (df_clean["user_rating"].between(0, 5)) &
    (df_clean["price"] >= 0) &
    (df_clean["reviews"] >= 0)
].copy()

df_clean.shape

(550, 7)

## Save Cleaned Dataset

In [30]:
clean_dir = repo_root / "data_cleaned"
clean_dir.mkdir(parents=True, exist_ok=True)

clean_path = clean_dir / "amazon_books_cleaned.csv"
df_clean.to_csv(clean_path, index=False)

print("Saved cleaned data to:", clean_path)
print("Clean shape:", df_clean.shape)


Saved cleaned data to: /workspaces/london-airbnb-market-expansion/data_cleaned/amazon_books_cleaned.csv
Clean shape: (550, 7)
